-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

## SparkML
In this notebook, we'll use Spark for:

* Sentiment Analysis
* Natural Language Processing (NLP)
* Decision Trees

We will be using a dataset of roughly 50,000 IMDB reviews, which includes the English language text of that review and the rating associated with it (1-10). Based on the text of the review, we want to predict if the rating is "positive" or "negative".

In [3]:
%run "./Includes/Classroom_Setup"

In [4]:
reviewsDF = spark.read.parquet("/mnt/training/movie-reviews/imdb/imdb_ratings_50k.parquet")
reviewsDF.createOrReplaceTempView("reviews")
display(reviewsDF)

review rating It's hard to imagine a director capable of such godawful crap as 'Notting Hill' pulling off something as sensitive and as attractive as this, but well, here's the evidence and it's quite compelling. Several have alluded to TV drama, and yes, this does have a seventies Play for Today feel at times, but is always a cut above, mainly I think owing to some quite superlative acting from Anne Reid and to a fine script which shadow-boxes with cliché without ever getting one on the nose, except maybe right at the end. (I didn't like either the tracking shot of indifferent goodbyes through the hallway, nor the oh-what-a-beautiful-morning final scene: she deserved a more studied finale than that I think, after all that hard work. The slippers business was a bit OTT too, on reflection). What I mean about avoiding cliché: well, I for one had a sinking expectation that the 'mature' man May's daughter tries to set her up with would be cast in 2 dimensions as a repulsive old bore, so as to point the contrast more painfully with the attractive, virile young geezer he is unwittingly competing with. Instead, we get an unexpectedly subtle and sympathetic cameo of a lonely, clumsy, not entirely unlikeable and very human fellow, who nevertheless doesn't have much of a clue about entertaining a woman. It was around that point I started to sit up and pay more attention. Here was a script that let the actors breathe and do something interesting with fairly minor parts. Almost Mike Leigh in that respect (minus the contrived catharses that the latter inexplicably goes in for). And of course I was, as everyone probably was, dumbfounded by what Anne Reid does with her character and with her body. She's /not/ 'the repressed, dutiful housewife discovering herself for the first time', this is far too simplistic for the character we have. Again and again there are allusions to her having been a 'bad housewife', not to mention that thing she does with trays, trying to look nurturing and comely and only succeeding in looking awkward. The daughter accuses her of having 'sat in front of the TV all day' instead of, well, whatever her motherly duties might be presumed to have been: she has no answer. She never was a model wife and mother, at least not to herself - that's where a lot of the poignancy comes from, the sense of someone having wasted a life trying to fulfil a role she simply wasn't good at, ever. 9.0 Two of Hollywood's great child stars (Elizabeth Taylor and Mickey Rooney) are perfectly teamed to deliver one of the all-time family classics. The story of a determined 12 year old girl, whose adoration for horses won't allow her to turn away from her goal to win the British Nationals. Mickey Rooney is the newly orphaned drifter, looking to tie himself over until he can follow his own dreams. A beautiful side plot reveals that the girl's mother had ambitions of her own as a young girl. Repeatedly overruling the father's decisions in favor of their spirited daughter, it is mother who seems to know best. The scene where the exhausted Taylor rushes to go to school is priceless: father protests 'why did you let her go to school, she'll drop from exhaustion before noon!' - Not to worry, she'd be back within a half hour; it was Saturday! The brilliant Technicolor, the sumptuous music score, and those beautiful faces, all telling a bittersweet story. Here's a good reason for the old saying: They don't make'm like that anymore! 10.0 I saw this film while I was in France and I must say that it confused me. It is a story of a jewel thief and a young singer who each end up in Morocco at the same time, run into one another and form a connection. Simple enough? Well, the problem is that this is the sort of film that has ambiguity in both chronology (the film is not played entirely in order) and in reality (did what i just saw really happen, or was it only a dream?). Given those parameters, as well as the film being bilingual, it was really hard to follow,

What does the distribution of scores look like?

HINT: Use `count()`

In [6]:
%sql
--ANSWER
SELECT count(rating), rating FROM reviews GROUP BY rating ORDER BY rating

count(rating) rating 10120 1.0 4586 2.0 4961 3.0 5331 4.0 4802 7.0 5858 8.0 4606 9.0 9730 10.0

The authors of this dataset have removed the "neutral" ratings, which they defined as a rating of 5 or 6.

## Train Test Split

We'll split our data into training and test samples. We will use 80% for training, and the remaining 20% for testing. We set a seed to reproduce the same results (i.e. if you re-run this notebook, you'll get the same results both times).

In [9]:
(trainDF, testDF) = reviewsDF.randomSplit([0.8, 0.2], seed=42)
trainDF.cache()
testDF.cache()

Out[ 23 ]: DataFrame[review: string, rating: double]

Let's determine our baseline accuracy

In [11]:
positiveRatings = trainDF.filter("rating >= 5").count()
totalRatings = trainDF.count()

print("Baseline accuracy: {0:.2f}%".format(positiveRatings/totalRatings*100))

Baseline accuracy: 50.01%

## Transformers + Estimators

In [13]:
from pyspark.ml.feature import RegexTokenizer

tokenizer = (RegexTokenizer()
            .setInputCol("review")
            .setOutputCol("tokens")
            .setPattern("\\W+"))

tokenizedDF = tokenizer.transform(reviewsDF)
display(tokenizedDF.limit(5)) # Look at a few tokenized reviews

review rating tokens It's hard to imagine a director capable of such godawful crap as 'Notting Hill' pulling off something as sensitive and as attractive as this, but well, here's the evidence and it's quite compelling. Several have alluded to TV drama, and yes, this does have a seventies Play for Today feel at times, but is always a cut above, mainly I think owing to some quite superlative acting from Anne Reid and to a fine script which shadow-boxes with cliché without ever getting one on the nose, except maybe right at the end. (I didn't like either the tracking shot of indifferent goodbyes through the hallway, nor the oh-what-a-beautiful-morning final scene: she deserved a more studied finale than that I think, after all that hard work. The slippers business was a bit OTT too, on reflection). What I mean about avoiding cliché: well, I for one had a sinking expectation that the 'mature' man May's daughter tries to set her up with would be cast in 2 dimensions as a repulsive old bore, so as to point the contrast more painfully with the attractive, virile young geezer he is unwittingly competing with. Instead, we get an unexpectedly subtle and sympathetic cameo of a lonely, clumsy, not entirely unlikeable and very human fellow, who nevertheless doesn't have much of a clue about entertaining a woman. It was around that point I started to sit up and pay more attention. Here was a script that let the actors breathe and do something interesting with fairly minor parts. Almost Mike Leigh in that respect (minus the contrived catharses that the latter inexplicably goes in for). And of course I was, as everyone probably was, dumbfounded by what Anne Reid does with her character and with her body. She's /not/ 'the repressed, dutiful housewife discovering herself for the first time', this is far too simplistic for the character we have. Again and again there are allusions to her having been a 'bad housewife', not to mention that thing she does with trays, trying to look nurturing and comely and only succeeding in looking awkward. The daughter accuses her of having 'sat in front of the TV all day' instead of, well, whatever her motherly duties might be presumed to have been: she has no answer. She never was a model wife and mother, at least not to herself - that's where a lot of the poignancy comes from, the sense of someone having wasted a life trying to fulfil a role she simply wasn't good at, ever. 9.0 List(it, s, hard, to, imagine, a, director, capable, of, such, godawful, crap, as, notting, hill, pulling, off, something, as, sensitive, and, as, attractive, as, this, but, well, here, s, the, evidence, and, it, s, quite, compelling, several, have, alluded, to, tv, drama, and, yes, this, does, have, a, seventies, play, for, today, feel, at, times, but, is, always, a, cut, above, mainly, i, think, owing, to, some, quite, superlative, acting, from, anne, reid, and, to, a, fine, script, which, shadow, boxes, with, clich, without, ever, getting, one, on, the, nose, except, maybe, right, at, the, end, i, didn, t, like, either, the, tracking, shot, of, indifferent, goodbyes, through, the, hallway, nor, the, oh, what, a, beautiful, morning, final, scene, she, deserved, a, more, studied, finale, than, that, i, think, after, all, that, hard, work, the, slippers, business, was, a, bit, ott, too, on, reflection, br, br, what, i, mean, about, avoiding, clich, well, i, for, one, had, a, sinking, expectation, that, the, mature, man, may, s, daughter, tries, to, set, her, up, with, would, be, cast, in, 2, dimensions, as, a, repulsive, old, bore, so, as, to, point, the, contrast, more, painfully, with, the, attractive, virile, young, geezer, he, is, unwittingly, competing, with, instead, we, get, an, unexpectedly, subtle, and, sympathetic, cameo, of, a, lonely, clumsy, not, entirely, unlikeable, and, very, human, fellow, who, nevertheless, doesn, t, have, much, of, a, clue, about, entertaining, a, woman, it, was, around, that, point, i, started, to, 

There are a lot of words that do not contain much information about the sentiment of the review (e.g. `the`, `a`, etc.). Let's remove these uninformative words using `StopWordsRemover`.

In [15]:
from pyspark.ml.feature import StopWordsRemover

remover = (StopWordsRemover()
          .setInputCol("tokens")
          .setOutputCol("stopWordFree"))

removedStopWordsDF = remover.transform(tokenizedDF)
display(removedStopWordsDF.limit(5)) # Look at a few tokenized reviews without stop words

review rating tokens stopWordFree It's hard to imagine a director capable of such godawful crap as 'Notting Hill' pulling off something as sensitive and as attractive as this, but well, here's the evidence and it's quite compelling. Several have alluded to TV drama, and yes, this does have a seventies Play for Today feel at times, but is always a cut above, mainly I think owing to some quite superlative acting from Anne Reid and to a fine script which shadow-boxes with cliché without ever getting one on the nose, except maybe right at the end. (I didn't like either the tracking shot of indifferent goodbyes through the hallway, nor the oh-what-a-beautiful-morning final scene: she deserved a more studied finale than that I think, after all that hard work. The slippers business was a bit OTT too, on reflection). What I mean about avoiding cliché: well, I for one had a sinking expectation that the 'mature' man May's daughter tries to set her up with would be cast in 2 dimensions as a repulsive old bore, so as to point the contrast more painfully with the attractive, virile young geezer he is unwittingly competing with. Instead, we get an unexpectedly subtle and sympathetic cameo of a lonely, clumsy, not entirely unlikeable and very human fellow, who nevertheless doesn't have much of a clue about entertaining a woman. It was around that point I started to sit up and pay more attention. Here was a script that let the actors breathe and do something interesting with fairly minor parts. Almost Mike Leigh in that respect (minus the contrived catharses that the latter inexplicably goes in for). And of course I was, as everyone probably was, dumbfounded by what Anne Reid does with her character and with her body. She's /not/ 'the repressed, dutiful housewife discovering herself for the first time', this is far too simplistic for the character we have. Again and again there are allusions to her having been a 'bad housewife', not to mention that thing she does with trays, trying to look nurturing and comely and only succeeding in looking awkward. The daughter accuses her of having 'sat in front of the TV all day' instead of, well, whatever her motherly duties might be presumed to have been: she has no answer. She never was a model wife and mother, at least not to herself - that's where a lot of the poignancy comes from, the sense of someone having wasted a life trying to fulfil a role she simply wasn't good at, ever. 9.0 List(it, s, hard, to, imagine, a, director, capable, of, such, godawful, crap, as, notting, hill, pulling, off, something, as, sensitive, and, as, attractive, as, this, but, well, here, s, the, evidence, and, it, s, quite, compelling, several, have, alluded, to, tv, drama, and, yes, this, does, have, a, seventies, play, for, today, feel, at, times, but, is, always, a, cut, above, mainly, i, think, owing, to, some, quite, superlative, acting, from, anne, reid, and, to, a, fine, script, which, shadow, boxes, with, clich, without, ever, getting, one, on, the, nose, except, maybe, right, at, the, end, i, didn, t, like, either, the, tracking, shot, of, indifferent, goodbyes, through, the, hallway, nor, the, oh, what, a, beautiful, morning, final, scene, she, deserved, a, more, studied, finale, than, that, i, think, after, all, that, hard, work, the, slippers, business, was, a, bit, ott, too, on, reflection, br, br, what, i, mean, about, avoiding, clich, well, i, for, one, had, a, sinking, expectation, that, the, mature, man, may, s, daughter, tries, to, set, her, up, with, would, be, cast, in, 2, dimensions, as, a, repulsive, old, bore, so, as, to, point, the, contrast, more, painfully, with, the, attractive, virile, young, geezer, he, is, unwittingly, competing, with, instead, we, get, an, unexpectedly, subtle, and, sympathetic, cameo, of, a, lonely, clumsy, not, entirely, unlikeable, and, very, human, fellow, who, nevertheless, doesn, t, have, much, of, a, clue, about, entertaining, a, woman, it, was, around, that, point, i, 

Where do the stop words actually come from? Spark includes a small English list as a default, which we're implicitly using here.

In [17]:
stopWords = remover.getStopWords()
stopWords

Out[ 27 ]: 
['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 'should',
 'now',
 "i'll",
 "you'll",
 "he'll",
 "she'll",
 "we'll",
 "they'll",
 "i'd",
 "you'd",
 "he'd",
 "she'd",
 "we'd",
 "they'd",
 "i'm",
 "you're",
 "he's",
 "she's",
 "it's",
 "we're",
 "they're",
 "i've",
 "we've",
 "you've",
 "they've",
 "isn't",
 "aren't",
 "wasn't",
 "weren't",
 "haven't",
 "hasn't",
 "hadn't",
 "don't",
 "doesn't",
 "didn't",
 "won't",
 "wouldn't",
 "shan't",
 "shouldn't",
 "mustn't",
 "can't",
 "couldn't",
 'cannot',
 'could',
 "here's",
 "how's",
 "let's",
 'ought',
 "that's",
 "there's",
 "what's",
 "when's",
 "where's",
 "who's",
 "why's",
 'would']

Let's remove the `br` from our reviews.

In [19]:
remover.setStopWords(["br"] + stopWords)
removedStopWordsDF = remover.transform(tokenizedDF)

Let's apply a CountVectorizer model

In [21]:
from pyspark.ml.feature import CountVectorizer

counts = (CountVectorizer()
          .setInputCol("stopWordFree")
          .setOutputCol("features")
          .setVocabSize(1000))

countModel = counts.fit(removedStopWordsDF) # It's a model

__Now let's adjust the label (target) values__

We want to group the reviews into "positive" or "negative" sentiment. So all of the star "levels" need to be collapsed into one of two groups.

In [23]:
from pyspark.ml.feature import Binarizer

binarizer = (Binarizer()
            .setInputCol("rating")
            .setOutputCol("label")
            .setThreshold(5.0))

Now we are going to use a Decision Tree model to fit to our dataset.

In [25]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline

dtc = DecisionTreeClassifier()

pipeline = Pipeline().setStages([tokenizer, remover, counts, binarizer, dtc])

In [26]:
pipelineModel = pipeline.fit(trainDF)
decisionTree = pipelineModel.stages[-1]

In [27]:
print(decisionTree.toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4d9485ddd9d91f79ff96) of depth 5 with 61 nodes
 If (feature 12 <= 0.5)
 If (feature 147 <= 0.5)
 If (feature 339 <= 0.5)
 If (feature 14 <= 0.5)
 If (feature 261 <= 0.5)
 Predict: 1.0
 Else (feature 261 > 0.5)
 Predict: 0.0
 Else (feature 14 > 0.5)
 If (feature 261 <= 0.5)
 Predict: 1.0
 Else (feature 261 > 0.5)
 Predict: 0.0
 Else (feature 339 > 0.5)
 If (feature 78 <= 2.5)
 If (feature 396 <= 0.5)
 Predict: 0.0
 Else (feature 396 > 0.5)
 Predict: 0.0
 Else (feature 78 > 2.5)
 If (feature 22 <= 1.5)
 Predict: 1.0
 Else (feature 22 > 1.5)
 Predict: 0.0
 Else (feature 147 > 0.5)
 If (feature 189 <= 1.5)
 If (feature 208 <= 0.5)
 If (feature 206 <= 0.5)
 Predict: 0.0
 Else (feature 206 > 0.5)
 Predict: 0.0
 Else (feature 208 > 0.5)
 If (feature 210 <= 0.5)
 Predict: 0.0
 Else (feature 210 > 0.5)
 Predict: 1.0
 Else (feature 189 > 1.5)
 If (feature 2 <= 1.5)
 If (feature 1 <= 4.5)
 Predict: 0.0
 Else (feature 1 > 4.5)
 Predict: 1.0
 Else (feature 2 > 1.5)
 If (feature 29 <= 1.5)
 Predict: 1.0
 Else (feature 29 > 1.5)
 Predict: 0.0
 Else (feature 12 > 0.5)
 If (feature 14 <= 0.5)
 If (feature 208 <= 0.5)
 If (feature 147 <= 0.5)
 If (feature 12 <= 1.5)
 Predict: 0.0
 Else (feature 12 > 1.5)
 Predict: 0.0
 Else (feature 147 > 0.5)
 If (feature 81 <= 4.5)
 Predict: 0.0
 Else (feature 81 > 4.5)
 Predict: 1.0
 Else (feature 208 > 0.5)
 If (feature 305 <= 0.5)
 If (feature 12 <= 1.5)
 Predict: 1.0
 Else (feature 12 > 1.5)
 Predict: 0.0
 Else (feature 305 > 0.5)
 If (feature 284 <= 0.5)
 Predict: 0.0
 Else (feature 284 > 0.5)
 Predict: 1.0
 Else (feature 14 > 0.5)
 If (feature 147 <= 0.5)
 If (feature 12 <= 1.5)
 If (feature 34 <= 0.5)
 Predict: 0.0
 Else (feature 34 > 0.5)
 Predict: 1.0
 Else (feature 12 > 1.5)
 If (feature 578 <= 0.5)
 Predict: 0.0
 Else (feature 578 > 0.5)
 Predict: 1.0
 Else (feature 147 > 0.5)
 If (feature 558 <= 2.5)
 If (feature 285 <= 0.5)
 Predict: 0.0
 Else (feature 285 > 0.5)
 Predict: 1.0
 Else (feature 558 > 2.5)
 Predict: 1.0

Let's save the pipeline model.

In [29]:
fileName = userhome + "/tmp/DT_Pipeline"
pipelineModel.write().overwrite().save(fileName)

In [30]:
from pyspark.ml import PipelineModel
# Load saved model
savedPipelineModel = PipelineModel.load(fileName)

In [31]:
resultDF = savedPipelineModel.transform(testDF)

#### Evaluate

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy: %(result)s" % {"result": evaluator.evaluate(resultDF)})

Accuracy: 0.6955121754880258

#### Confusion Matrix

In [35]:
display(resultDF.groupBy("label", "prediction").count())

label prediction count 1.0 1.0 4339 0.0 1.0 2403 1.0 0.0 623 0.0 0.0 2573

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>